In [97]:
from selenium import webdriver
from datetime import datetime
import pandas as pd
import numpy as np
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException


# Set up the Selenium WebDriver (make sure to have the appropriate webdriver installed)
driver = webdriver.Edge(executable_path=r"C:\Program Files (x86)\msedgedriver.exe")


# URL of the webpage containing the dropdown menu
url = f"https://www.basketball-reference.com/leagues/NBA_2024_per_game.html"
driver.get(url)

# Player Ratings

In [100]:
# Have the table only show accurate data
hide_partial_row_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '//*[@id="per_game_stats_toggle_partial_table"]'))
)

hide_partial_row_button.click()
print('Hide rows with partial data')

Hide rows with partial data


In [101]:
# Get table with data
table = driver.find_element(By.XPATH,'//*[@id="per_game_stats"]')

# Get table header
headers = table.find_element(By.XPATH, '//*[@id="per_game_stats"]/thead').text.split(' ')

# Get data from table
tbody = table.find_element(By.XPATH, '//*[@id="per_game_stats"]/tbody')
rows = [row.text.split(' ') for row in tbody.find_elements(By.CLASS_NAME, 'full_table')]

In [93]:
data = []

for row in rows:
    # Find rows where the team name has two parts 'Boston Celtics'
    if len(row) == 31:
        row.pop(1)
        row.pop(1)
    # Find rows where the team name has three parts 'Oklahoma City Thunder'
    elif len(row) == 32:
        row.pop(1)
        row.pop(1)
        row.pop(1)
    # elif row[0] == 'Unadjusted' or row[0] == 'Rk':
    #     continue  # Skip headers and other irrelevant rows
    data.append(row)
    
# Close the browser
driver.close()
    
data

[['1',
  'C-PF',
  '24',
  'TOT',
  '31',
  '0',
  '16.5',
  '2.8',
  '6.1',
  '.458',
  '0.5',
  '1.7',
  '.269',
  '2.4',
  '4.5',
  '.529',
  '.495',
  '0.8',
  '1.4',
  '.568',
  '2.0',
  '3.2',
  '5.2',
  '1.5',
  '0.6',
  '0.5',
  '1.1',
  '1.6',
  '6.9'],
 ['2',
  'C',
  '26',
  'MIA',
  '27',
  '27',
  '34.4',
  '8.0',
  '15.7',
  '.513',
  '0.0',
  '0.3',
  '.143',
  '8.0',
  '15.4',
  '.519',
  '.514',
  '6.0',
  '7.6',
  '.782',
  '2.2',
  '8.1',
  '10.4',
  '4.0',
  '1.1',
  '1.0',
  '2.5',
  '2.4',
  '22.1'],
 ['3',
  'SG',
  '23',
  'UTA',
  '39',
  '10',
  '20.3',
  '2.4',
  '5.3',
  '.444',
  '1.1',
  '3.1',
  '.339',
  '1.3',
  '2.2',
  '.593',
  '.543',
  '0.3',
  '0.4',
  '.733',
  '0.8',
  '1.7',
  '2.5',
  '1.0',
  '0.6',
  '0.5',
  '0.7',
  '1.5',
  '6.1'],
 ['4',
  'PF',
  '23',
  'MEM',
  '29',
  '5',
  '23.2',
  '3.9',
  '9.0',
  '.438',
  '1.6',
  '4.6',
  '.341',
  '2.4',
  '4.4',
  '.539',
  '.525',
  '1.0',
  '1.6',
  '.609',
  '1.3',
  '4.2',
  '5.5',
  '1

In [132]:
player_names = []

# Extract team names from table
names = tbody.find_elements(By.CLASS_NAME, 'left')

# Extract team names and add to array
for name in names:
    if name.text == '' or len(name.text) >= 3:
        continue
    else:
        player_names.append(name.text)

player_names

['Precious Achiuwa',
 'TOT',
 '',
 '',
 '',
 '',
 'Bam Adebayo',
 'MIA',
 'Ochai Agbaji',
 'UTA',
 'Santi Aldama',
 'MEM',
 'Nickeil Alexander-Walker',
 'MIN',
 'Grayson Allen',
 'PHO',
 'Jarrett Allen',
 'CLE',
 'Jose Alvarado',
 'NOP',
 'Kyle Anderson',
 'MIN',
 'Giannis Antetokounmpo',
 'MIL',
 'Thanasis Antetokounmpo',
 'MIL',
 'Cole Anthony',
 'ORL',
 'OG Anunoby',
 'TOT',
 '',
 '',
 '',
 '',
 'Ryan Arcidiacono',
 'NYK',
 'Deni Avdija',
 'WAS',
 'Deandre Ayton',
 'POR',
 'Udoka Azubuike',
 'PHO',
 'Ibou Badji',
 'POR',
 'Marvin Bagley III',
 'DET',
 'Amari Bailey',
 'CHO',
 'Patrick Baldwin Jr.',
 'WAS',
 'LaMelo Ball',
 'CHO',
 'Mo Bamba',
 'PHI',
 'Paolo Banchero',
 'ORL',
 'Desmond Bane',
 'MEM',
 'Dalano Banton',
 'BOS',
 'Dominick Barlow',
 'SAS',
 'Harrison Barnes',
 'SAC',
 'Scottie Barnes',
 'TOR',
 'RJ Barrett',
 'TOT',
 '',
 '',
 '',
 '',
 'Charles Bassey',
 'SAS',
 'Emoni Bates',
 'CLE',
 'Keita Bates-Diop',
 'PHO',
 'Nicolas Batum',
 'TOT',
 '',
 '',
 '',
 '',
 'Bradle

In [64]:
# Add respective team names to corresponding table data rows
for i in range(len(data)):
    data[i].insert(1, player_names[i])

IndexError: list index out of range

In [6]:
# Remove 'Player' column from list
headers.pop(1)

# Add 'First Name' to list
headers.insert(1, 'First Name')

# Add 'Last Name' to list
headers.insert(2, 'Last Name')

driver.close()

In [7]:
# Create a pandas Data Frame
player_df = pd.DataFrame(data=data)

# Drop last column
player_df = player_df.drop(player_df.columns[31],axis=1)

# Add headers
player_df.columns = headers

In [8]:
player_df.head()

,Rk,First Name,Last Name,Pos,Age,Tm,G,GS,MP,FG,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1,Precious,Achiuwa,C-PF,24,TOT,31,0,16.5,2.8,...,.568,2.0,3.2,5.2,1.5,0.6,0.5,1.1,1.6,6.9
1,,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,2,Bam,Adebayo,C,26,MIA,27,27,34.4,8.0,...,.782,2.2,8.1,10.4,4.0,1.1,1.0,2.5,2.4,22.1
4,3,Ochai,Agbaji,SG,23,UTA,39,10,20.3,2.4,...,.733,0.8,1.7,2.5,1.0,0.6,0.5,0.7,1.5,6.1


In [9]:
player_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 576 entries, 0 to 575
Data columns (total 31 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Rk          576 non-null    object
 1   First Name  548 non-null    object
 2   Last Name   548 non-null    object
 3   Pos         548 non-null    object
 4   Age         548 non-null    object
 5   Tm          548 non-null    object
 6   G           548 non-null    object
 7   GS          548 non-null    object
 8   MP          548 non-null    object
 9   FG          548 non-null    object
 10  FGA         548 non-null    object
 11  FG%         548 non-null    object
 12  3P          548 non-null    object
 13  3PA         548 non-null    object
 14  3P%         548 non-null    object
 15  2P          548 non-null    object
 16  2PA         548 non-null    object
 17  2P%         548 non-null    object
 18  eFG%        548 non-null    object
 19  FT          548 non-null    object
 20  FTA       

In [10]:
# Remove rows with any null values'
player_df = player_df.dropna(axis=0, how='any')

In [11]:
player_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 445 entries, 0 to 574
Data columns (total 31 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Rk          445 non-null    object
 1   First Name  445 non-null    object
 2   Last Name   445 non-null    object
 3   Pos         445 non-null    object
 4   Age         445 non-null    object
 5   Tm          445 non-null    object
 6   G           445 non-null    object
 7   GS          445 non-null    object
 8   MP          445 non-null    object
 9   FG          445 non-null    object
 10  FGA         445 non-null    object
 11  FG%         445 non-null    object
 12  3P          445 non-null    object
 13  3PA         445 non-null    object
 14  3P%         445 non-null    object
 15  2P          445 non-null    object
 16  2PA         445 non-null    object
 17  2P%         445 non-null    object
 18  eFG%        445 non-null    object
 19  FT          445 non-null    object
 20  FTA       

In [12]:
# Convert columns with data type object to float
player_df = player_df.apply(pd.to_numeric, errors='ignore', downcast='float')
player_df.dtypes

Rk            float32
First Name     object
Last Name      object
Pos            object
Age            object
Tm             object
G              object
GS            float32
MP            float32
FG            float32
FGA           float32
FG%           float32
3P            float32
3PA           float32
3P%           float32
2P            float32
2PA           float32
2P%           float32
eFG%          float32
FT            float32
FTA           float32
FT%           float32
ORB           float32
DRB           float32
TRB           float32
AST           float32
STL           float32
BLK           float32
TOV           float32
PF            float32
PTS           float32
dtype: object

In [13]:
player_df

,Rk,First Name,Last Name,Pos,Age,Tm,G,GS,MP,FG,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1.0,Precious,Achiuwa,C-PF,24,TOT,31,0.0,16.500000,2.8,...,0.568,2.0,3.2,5.2,1.5,0.6,0.5,1.1,1.6,6.900000
3,2.0,Bam,Adebayo,C,26,MIA,27,27.0,34.400002,8.0,...,0.782,2.2,8.1,10.4,4.0,1.1,1.0,2.5,2.4,22.100000
4,3.0,Ochai,Agbaji,SG,23,UTA,39,10.0,20.299999,2.4,...,0.733,0.8,1.7,2.5,1.0,0.6,0.5,0.7,1.5,6.100000
5,4.0,Santi,Aldama,PF,23,MEM,29,5.0,23.200001,3.9,...,0.609,1.3,4.2,5.5,1.7,0.7,0.7,1.1,1.6,10.400000
6,5.0,Nickeil,Alexander-Walker,SG,25,MIN,37,12.0,22.400000,2.4,...,0.696,0.5,1.3,1.8,2.4,0.9,0.6,0.9,1.9,6.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569,517.0,Delon,Wright,PG,31,WAS,16,0.0,17.000000,1.5,...,0.783,0.4,1.9,2.3,3.3,1.4,0.2,0.6,0.8,4.600000
570,518.0,Thaddeus,Young,PF,35,TOR,10,2.0,8.700000,1.6,...,0.333,0.6,1.0,1.6,1.4,0.7,0.1,0.7,0.8,3.400000
571,519.0,Trae,Young,PG,25,ATL,34,34.0,36.599998,8.6,...,0.855,0.4,2.6,3.0,11.0,1.4,0.2,4.3,2.0,27.799999
572,520.0,Omer,Yurtseven,C,25,UTA,22,6.0,11.300000,1.6,...,0.600,1.5,2.9,4.4,0.6,0.2,0.5,1.0,1.5,3.700000


In [24]:
player_df.to_csv('player_ratings.csv', sep='\t', encoding='utf-8', index=False)

# Team Ratings

In [14]:
from selenium import webdriver
from datetime import datetime
import pandas as pd
import numpy as np
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException

# Set up the Selenium WebDriver (make sure to have the appropriate webdriver installed)
driver = webdriver.Edge(executable_path=r"C:\Program Files (x86)\msedgedriver.exe")

# URL of the webpage containing the dropdown menu
url = f"https://www.basketball-reference.com/leagues/NBA_2024_ratings.html"
driver.get(url)

In [15]:
# Extract table
table = WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="ratings"]'))
)

# Get headers used in table
thead = table.find_element(By.TAG_NAME, 'thead')
headers = [header.text.split() for header in thead.find_elements(By.TAG_NAME,'tr')][1]

# Get data from table
tbody = table.find_element(By.XPATH, '//*[@id="ratings"]/tbody')
rows = [row.text.split() for row in tbody.find_elements(By.TAG_NAME, 'tr')]

In [17]:
data = []

for row in rows:
    # Find rows where the team name has two parts 'Boston Celtics'
    if len(row) == 16:
        row.pop(1)
        row.pop(1)
    # Find rows where the team name has three parts 'Oklahoma City Thunder'
    elif len(row) == 17:
        row.pop(1)
        row.pop(1)
        row.pop(1)
    elif row[0] == 'Unadjusted' or row[0] == 'Rk':
        continue  # Skip headers and other irrelevant rows
    data.append(row)
    
# Close the browser
driver.close()
    
data

[['1',
  'E',
  'A',
  '29',
  '9',
  '.763',
  '9.21',
  '122.45',
  '113.05',
  '9.41',
  '9.92',
  '122.60',
  '112.49',
  '10.11'],
 ['2',
  'W',
  'NW',
  '26',
  '11',
  '.703',
  '9.05',
  '122.60',
  '113.54',
  '9.06',
  '8.99',
  '122.72',
  '113.72',
  '9.00'],
 ['3',
  'E',
  'A',
  '23',
  '13',
  '.639',
  '7.81',
  '120.61',
  '112.78',
  '7.83',
  '7.04',
  '119.99',
  '112.92',
  '7.07'],
 ['4',
  'W',
  'NW',
  '26',
  '11',
  '.703',
  '5.38',
  '115.60',
  '110.14',
  '5.46',
  '6.62',
  '115.96',
  '109.26',
  '6.71'],
 ['5',
  'W',
  'P',
  '24',
  '13',
  '.649',
  '5.65',
  '120.75',
  '115.04',
  '5.70',
  '5.18',
  '120.47',
  '115.23',
  '5.24'],
 ['6',
  'W',
  'NW',
  '26',
  '13',
  '.667',
  '4.95',
  '120.04',
  '114.88',
  '5.15',
  '4.86',
  '120.36',
  '115.32',
  '5.04'],
 ['7',
  'W',
  'SW',
  '23',
  '15',
  '.605',
  '5.00',
  '118.35',
  '113.23',
  '5.12',
  '4.91',
  '118.82',
  '113.80',
  '5.02'],
 ['8',
  'E',
  'A',
  '22',
  '16',
  '.579

In [ ]:
team_names = []

# Extract team names from table
names = tbody.find_elements(By.CLASS_NAME, 'left')

# Extract team names and add to array
for name in names:
    if name.text == 'Team':
        continue
    else:
        team_names.append(name.text)

team_names

In [18]:
# Add respective team names to corresponding table data rows
for i in range(len(data)):
    data[i].insert(1, team_names[i])

In [19]:
data

[['1',
  'Boston Celtics',
  'E',
  'A',
  '29',
  '9',
  '.763',
  '9.21',
  '122.45',
  '113.05',
  '9.41',
  '9.92',
  '122.60',
  '112.49',
  '10.11'],
 ['2',
  'Oklahoma City Thunder',
  'W',
  'NW',
  '26',
  '11',
  '.703',
  '9.05',
  '122.60',
  '113.54',
  '9.06',
  '8.99',
  '122.72',
  '113.72',
  '9.00'],
 ['3',
  'Philadelphia 76ers',
  'E',
  'A',
  '23',
  '13',
  '.639',
  '7.81',
  '120.61',
  '112.78',
  '7.83',
  '7.04',
  '119.99',
  '112.92',
  '7.07'],
 ['4',
  'Minnesota Timberwolves',
  'W',
  'NW',
  '26',
  '11',
  '.703',
  '5.38',
  '115.60',
  '110.14',
  '5.46',
  '6.62',
  '115.96',
  '109.26',
  '6.71'],
 ['5',
  'Los Angeles Clippers',
  'W',
  'P',
  '24',
  '13',
  '.649',
  '5.65',
  '120.75',
  '115.04',
  '5.70',
  '5.18',
  '120.47',
  '115.23',
  '5.24'],
 ['6',
  'Denver Nuggets',
  'W',
  'NW',
  '26',
  '13',
  '.667',
  '4.95',
  '120.04',
  '114.88',
  '5.15',
  '4.86',
  '120.36',
  '115.32',
  '5.04'],
 ['7',
  'New Orleans Pelicans',
  '

In [20]:
team_ratings = pd.DataFrame(data=data, columns=headers)
team_ratings

,Rk,Team,Conf,Div,W,L,W/L%,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A
0,1,Boston Celtics,E,A,29,9,.763,9.21,122.45,113.05,9.41,9.92,122.60,112.49,10.11
1,2,Oklahoma City Thunder,W,NW,26,11,.703,9.05,122.60,113.54,9.06,8.99,122.72,113.72,9.00
2,3,Philadelphia 76ers,E,A,23,13,.639,7.81,120.61,112.78,7.83,7.04,119.99,112.92,7.07
3,4,Minnesota Timberwolves,W,NW,26,11,.703,5.38,115.60,110.14,5.46,6.62,115.96,109.26,6.71
4,5,Los Angeles Clippers,W,P,24,13,.649,5.65,120.75,115.04,5.70,5.18,120.47,115.23,5.24
5,6,Denver Nuggets,W,NW,26,13,.667,4.95,120.04,114.88,5.15,4.86,120.36,115.32,5.04
6,7,New Orleans Pelicans,W,SW,23,15,.605,5.00,118.35,113.23,5.12,4.91,118.82,113.80,5.02
7,8,New York Knicks,E,A,22,16,.579,3.79,119.53,115.50,4.02,3.88,119.16,115.05,4.11
8,9,Milwaukee Bucks,E,C,26,12,.684,4.97,122.19,117.28,4.90,4.10,121.31,117.26,4.05
9,10,Indiana Pacers,E,C,22,15,.595,2.78,123.57,120.80,2.76,2.53,123.64,121.12,2.53


In [21]:
team_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Rk      30 non-null     object
 1   Team    30 non-null     object
 2   Conf    30 non-null     object
 3   Div     30 non-null     object
 4   W       30 non-null     object
 5   L       30 non-null     object
 6   W/L%    30 non-null     object
 7   MOV     30 non-null     object
 8   ORtg    30 non-null     object
 9   DRtg    30 non-null     object
 10  NRtg    30 non-null     object
 11  MOV/A   30 non-null     object
 12  ORtg/A  30 non-null     object
 13  DRtg/A  30 non-null     object
 14  NRtg/A  30 non-null     object
dtypes: object(15)
memory usage: 3.6+ KB


In [ ]:
# Convert selected columns from object to float
team_ratings = team_ratings.apply(pd.to_numeric, errors='ignore', downcast='float')
team_ratings.dtypes

In [23]:
team_ratings.to_csv('team_ratings.csv', sep='\t', encoding='utf-8', index=False)